## Preprocessing Data

In the following steps, the raw data was preprocessed for various regression models. To avoid influencing model performance, only model-independent preprocessing steps were applied.

### Setup

In [137]:
import pandas as pd
import numpy as np
import re
from sklearn.linear_model import LinearRegression       
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

### Import Data

In [138]:
# im port the CSV file data\raw_data.csv
df = pd.read_csv('../data/raw/raw_data.csv')
df.head() #Display the first 5 rows of the DataFramed

,product,isoelectric_point,protein_format,molecular_weight_da,formulation_title,composition,product_conc_mg_ml,tm_c
0,MAB5410990,6.162,IGG3,391603.56,F01,"15 mM Succinate + 67.5 mM L-Lysine + 67.5 mM KCl + 0.4 mg/mL PS50, pH 4.5",172.8,139.411938
1,MAB5410990,6.162,IGG3,391603.56,F02,"15 mM Succinate + 135 mM L-Lysine + 0.4 mg/mL PS50, pH 4.5",172.8,138.958645
2,MAB5410990,6.162,IGG3,391603.56,F03,"15 mM Succinate + 67.5 mM L-Lysine + 135 mM Mannitol + 0.4 mg/mL PS50, pH 4.5",172.8,143.647805
3,MAB5410990,6.162,IGG3,391603.56,F04,"15 mM Succinate + 135 mM KCl + 0.4 mg/mL PS50, pH 4.5",172.8,140.801897
4,MAB5410990,6.162,IGG3,391603.56,F05,"15 mM Succinate + 270 mM Mannitol + 0.4 mg/mL PS50, pH 4.5",172.8,151.632333


In [139]:
df.info()  # Display the DataFrame information to check the data types and non-null counts

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1210 entries, 0 to 1209
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   product              1210 non-null   object 
 1   isoelectric_point    1151 non-null   float64
 2   protein_format       1210 non-null   object 
 3   molecular_weight_da  1210 non-null   float64
 4   formulation_title    1210 non-null   object 
 5   composition          1210 non-null   object 
 6   product_conc_mg_ml   1210 non-null   float64
 7   tm_c                 1210 non-null   float64
dtypes: float64(4), object(4)
memory usage: 75.8+ KB


### Data Corrections

In [140]:
# For a better overview data set is reduced to the most interessting variables we want to examine. Therefore id columns are dropped.
df = df.drop(columns=['product', 'formulation_title'])

In [141]:
# change data type in order to make pandas functions more efficient
df['protein_format'] = pd.Categorical(df['protein_format'])

In [142]:
# Eliminate spacesin all column names
df.columns = df.columns.str.replace(' ', '_')   
# Make sure column names are lower case a
df.columns = df.columns.str.lower()

### Preprocessing String Data into Numerical Features

In [143]:
#Create new column ph
df['ph'] = df['composition'].str.split(',').str[-1].str.extract(r'(\d+)').astype(float)

In [144]:
#Create new column Excipients
df['composition_without_ph'] = df['composition'].str.split(',').str[0]
df.drop('composition', axis=1, inplace=True)

In [145]:
# Create a list to hold unique excipients 
excipients = []

# For every value in the column 'composition_without_ph', split the string by '+' and strip whitespace and put them into a list
for composition_without_ph in df['composition_without_ph']:
    list_conc_excipient = [c.strip() for c in composition_without_ph.split('+')] 
    # For every value in the list, split the string by ' ' and take the last part as the excipient name
    for conc_excipient in list_conc_excipient:
        parts = conc_excipient.split()
        if parts:  # Ignore empty strings
            excipients.append(parts[-1])

unique_excipients = list(set(excipients))  # Get unique excipients
print(unique_excipients)

['Mannitol', 'Succinate', 'Fructose', 'Citrate', 'KCl', 'PS50', 'PS80', 'L-Lysine']


In [146]:
# Function to extract the concentration of a specific excipient from a string
def extract_value(excipient_str,excipient):
    # Suche nach dem KCl-Teil in der Zeichenkette
    list_conc_excipient = [c.strip() for c in excipient_str.split('+')]
    for conc_excipient in list_conc_excipient:
        if excipient in conc_excipient:
            # Extrahiere den numerischen Wert
            match = re.search(r'(\d+(\.\d+)?)', conc_excipient)
            if match:
                return float(match.group(1))  
    return 0  # Wenn kein Wert gefunden wurde, None zurückgeben

In [147]:
# Create new columns for each unique excipient and extract their concentrations
for excipient in unique_excipients:
    new_column_name = excipient + '_conc' 
    # replace all uppercase letters with lowercase letters in the new column name
    new_column_name = new_column_name.lower()
    # Apply the extract_value function to the 'composition_without_ph' column for each excipient
    df[new_column_name] = df['composition_without_ph'].apply(lambda x: extract_value(x, excipient))
# Drop the 'composition_without_ph' column as it is no longer needed
df.drop('composition_without_ph', axis=1, inplace=True)


pd.set_option('display.max_colwidth', None)
df.head()  # Display the first 5 rows of the DataFrame with new columns for excipients



,isoelectric_point,protein_format,molecular_weight_da,product_conc_mg_ml,tm_c,ph,mannitol_conc,succinate_conc,fructose_conc,citrate_conc,kcl_conc,ps50_conc,ps80_conc,l-lysine_conc
0,6.162,IGG3,391603.56,172.8,139.411938,4.0,0.0,15.0,0.0,0.0,67.5,0.4,0.0,67.5
1,6.162,IGG3,391603.56,172.8,138.958645,4.0,0.0,15.0,0.0,0.0,0.0,0.4,0.0,135.0
2,6.162,IGG3,391603.56,172.8,143.647805,4.0,135.0,15.0,0.0,0.0,0.0,0.4,0.0,67.5
3,6.162,IGG3,391603.56,172.8,140.801897,4.0,0.0,15.0,0.0,0.0,135.0,0.4,0.0,0.0
4,6.162,IGG3,391603.56,172.8,151.632333,4.0,270.0,15.0,0.0,0.0,0.0,0.4,0.0,0.0


### Missing values handling

In [148]:
# Find all missing values in the DataFrame
missing_values = df.isnull().sum()
# Display the missing values
missing_values

isoelectric_point      59
protein_format          0
molecular_weight_da     0
product_conc_mg_ml      0
tm_c                    0
ph                      0
mannitol_conc           0
succinate_conc          0
fructose_conc           0
citrate_conc            0
kcl_conc                0
ps50_conc               0
ps80_conc               0
l-lysine_conc           0
dtype: int64

In [149]:
#Error handling for missing values in column 'isoelectric_point' by linear regression 
train_df = df[df['isoelectric_point'].notna()]
X_train = train_df[['molecular_weight_da']]
y_train = train_df['isoelectric_point']

print(f"Trainingsdaten: {X_train.shape[0]} Zeilen")  # Debug-Ausgabe

# Modell trainieren
model = LinearRegression()
model.fit(X_train, y_train)

# Maske für fehlende isoelectric_point Werte
mask_missing = df['isoelectric_point'].isna()

print(f"Zu prognostizierende Zeilen: {mask_missing.sum()}")  # Debug-Ausgabe

# Nur vorhersagen, wenn es auch Daten gibt
if mask_missing.sum() > 0:
    df.loc[mask_missing, 'isoelectric_point'] = model.predict(df.loc[mask_missing, ['molecular_weight_da']])
else:
    print("Keine Zeilen zum Vorhersagen – alle Werte vorhanden.")

# Find all missing values in the DataFrame
missing_values = df.isnull().sum()
# Display the missing values
missing_values


Trainingsdaten: 1151 Zeilen
Zu prognostizierende Zeilen: 59


isoelectric_point      0
protein_format         0
molecular_weight_da    0
product_conc_mg_ml     0
tm_c                   0
ph                     0
mannitol_conc          0
succinate_conc         0
fructose_conc          0
citrate_conc           0
kcl_conc               0
ps50_conc              0
ps80_conc              0
l-lysine_conc          0
dtype: int64

### Encoding Categorical Variables

In [150]:
# Create dummy variables (dummy columns are uint8 by default)
df = pd.get_dummies(df, columns=['protein_format'], drop_first=True)

# Identify only the dummy columns
dummy_cols = [col for col in df.columns if col.startswith('protein_format_')]

# Convert only those dummy columns to int type
df[dummy_cols] = df[dummy_cols].astype(int)

# Make sure column names are lower case a
df.columns = df.columns.str.lower()


In [151]:
df.info()  # Display the DataFrame information to check the data types and non-null counts

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1210 entries, 0 to 1209
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   isoelectric_point         1210 non-null   float64
 1   molecular_weight_da       1210 non-null   float64
 2   product_conc_mg_ml        1210 non-null   float64
 3   tm_c                      1210 non-null   float64
 4   ph                        1210 non-null   float64
 5   mannitol_conc             1210 non-null   float64
 6   succinate_conc            1210 non-null   float64
 7   fructose_conc             1210 non-null   float64
 8   citrate_conc              1210 non-null   float64
 9   kcl_conc                  1210 non-null   float64
 10  ps50_conc                 1210 non-null   float64
 11  ps80_conc                 1210 non-null   float64
 12  l-lysine_conc             1210 non-null   float64
 13  protein_format_igg3       1210 non-null   int64  
 14  protein_

### Export Preprocessed Data

In [152]:
#export the DataFrame to a new CSV file and overwrite the existing one 
df.to_csv('../data/processed/preprocessed_data.csv', index=False)